### Import libraries

In [3]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.data
import nltk
import glob
#from glob import glob
import os
import re
import string
import pandas as pd

In [9]:
#nltk.download('stopwords')
#nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kathrynkundrod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kathrynkundrod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Capture text from pdf files, a singular example
https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [12]:

filename = "Austin_Transcripts/scrape2/2015_01_29.pdf"

pdfFileObj = open(filename,'rb')

#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
count = 0
text = ""

#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()

#print(text)

## Capture text from pdf files, loop through all files

In [131]:
# 'glob' files, i.e., make a list of all with the same desired attribute
# in this case, all pdfs in the folder Austin_Transcripts
filelist = glob.glob("Austin_Transcripts/*.pdf")
#filelist

In [130]:
# for each file in filelist...
for pdffile in filelist:
    
    # Open the pdf file and make it a pdf object
    pdfFileObj = open(pdffile,'rb')

    # The pdfReader variable is a readable object that will be parsed
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # Discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""

    # This while loop will read each page
    # The 'text' output is what we want to save for each pdf
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()

    # Create an output file and put "text" in there
    file = open(output_filename, "w")
    file.write(text)
    fileroot, fileext = os.path.splitext(pdffile)
    output_filename = fileroot+'.txt'
    file.close()   


KeyboardInterrupt: 

## Cleaning text files, a singular example

In [67]:
# name the file before opening so it's easier to iterate on later
text_ex =  "Austin_Transcripts/2017_03_23.txt"
f = open(text_ex, 'r')

# turns f file into a string
content = f.read()
#type(content)   # it's a string
#print(content)  # see, it's a string

# always close files after pulling out the "content" or whatever you want to avoid overwriting
f.close()

In [68]:
## Basic text cleanup

# remove time stamp
# this re command removes everything between "[]"
text_cleaning = re.sub("[\[].*?[\]]", "", content)

# remove the new line instances
text_cleaning = text_cleaning.replace("\n", " ")

# subbing repeated white space for a single space
text_cleaning = re.sub('\s\s+', ' ', text_cleaning)

# remove header
text_cleaning = re.sub(r'.*====', '=', text_cleaning)

# remove all punctuation that is not ">" or ":"
# we need these puncts to parse speakers
remove = string.punctuation
remove = remove.replace(">", "") # don't remove carats
remove = remove.replace(":", "") # keep these too, which helps me see when CC members are speaking
pattern = r"[{}]".format(remove) # create the pattern
text_cleaning = re.sub(pattern, "", text_cleaning) 

## Split text by speaker. Each ">>" indicates a new speaker
# split by speakers - now I have a list of string, each string a new speaker
speaker_strings = text_cleaning.split(">>")
speaker_strings[10:20]
#type(speaker_strings)

## Now, remove strings said by a CCM or Mayor, i.e., those tagged with a name  
# initializing substring
# want to recognize [any alpha character + :]
subs = '\w+:' # "\w" indicates any alpha character 
  
# using list comprehension to return strings without the substring above
# I want to print all lists that DO NOT contain the substring
#speaker_strings2 = [i for i in speaker_strings1 if not subs in i] 
speaker_strings = [string for string in speaker_strings if not re.search(subs, string)]

# now, same thing to remove the prayer
subs2 = "Amen"
speaker_strings = [string for string in speaker_strings if not re.search(subs2, string)]

speaker_strings_long = []
for list_element in speaker_strings:
    if len(list_element) >= 100:
        speaker_strings_long.append(list_element)
else:
    pass

# concatenate into a string for eventual export
speaker_strings_long = '_'.join(speaker_strings_long)

## Now, loop through all files to clean and parse by speaker

In [135]:
# making a list of all txt files, to run through loop later
txt_list = glob.glob("Austin_Transcripts/*.txt")
txt_list

['Austin_Transcripts/2015-02-26.txt',
 'Austin_Transcripts/2015-03-26.txt',
 'Austin_Transcripts/2015-04-23.txt',
 'Austin_Transcripts/2015-05-21.txt',
 'Austin_Transcripts/2015-06-18.txt',
 'Austin_Transcripts/2015-08-20.txt',
 'Austin_Transcripts/2015-09-17.txt',
 'Austin_Transcripts/2015-10-15.txt',
 'Austin_Transcripts/2015-11-19.txt',
 'Austin_Transcripts/2015-12-17.txt',
 'Austin_Transcripts/2016-01-28.txt',
 'Austin_Transcripts/2016-02-25.txt',
 'Austin_Transcripts/2016-03-24.txt',
 'Austin_Transcripts/2016-04-21.txt',
 'Austin_Transcripts/2016-05-19.txt',
 'Austin_Transcripts/2016-06-23.txt',
 'Austin_Transcripts/2016-08-18.txt',
 'Austin_Transcripts/2016-09-22.txt',
 'Austin_Transcripts/2016-10-20.txt',
 'Austin_Transcripts/2016-11-10.txt',
 'Austin_Transcripts/2016-12-08.txt',
 'Austin_Transcripts/2016-12-15.txt',
 'Austin_Transcripts/2017-02-16.txt',
 'Austin_Transcripts/2017-03_23.txt',
 'Austin_Transcripts/2017-04-13.txt',
 'Austin_Transcripts/2017-05-18.txt',
 'Austin_Tra

In [59]:
#fileroot, fileext = os.path.splitext(raw_text)
#output_filename = fileroot+'clean_.txt'

In [136]:
for raw_text in txt_list:
    
    f = open(raw_text,'r')
    #text_ex =  "Austin_Transcripts/2017_03_23.txt"
    #f = open(text_ex, 'r')

    # turns f file into a string
    content = f.read()
    #type(content)   # it's a string
    #print(content)  # see, it's a string

    # always close files after pulling out the "content" or whatever you want to avoid overwriting
    f.close()
    
    # basic text cleaning
    text_cleaning = re.sub("[\[].*?[\]]", "", content)
    text_cleaning = text_cleaning.replace("\n", " ")
    text_cleaning = re.sub('\s\s+', ' ', text_cleaning)
    text_cleaning = re.sub(r'.*====', '=', text_cleaning)
    remove = string.punctuation
    remove = remove.replace(">", "") # don't remove carats
    remove = remove.replace(":", "") # keep these too, which helps me see when CC members are speaking
    pattern = r"[{}]".format(remove) # create the pattern
    text_cleaning = re.sub(pattern, "", text_cleaning) 

    # separate speakers
    speaker_strings = text_cleaning.split(">>")
    
    ## Now, remove strings said by a CCM or Mayor, i.e., those tagged with a name  
    # initializing substring
    # want to recognize [any alpha character + :]
    subs = '\w+:' # "\w" indicates any alpha character 

    # using list comprehension to return strings without the substring above
    # I want to print all lists that DO NOT contain the substring
    #speaker_strings2 = [i for i in speaker_strings1 if not subs in i] 
    speaker_strings = [string for string in speaker_strings if not re.search(subs, string)]

    # now, same thing to remove the prayer
    subs2 = "Amen"
    speaker_strings = [string for string in speaker_strings if not re.search(subs2, string)]

    speaker_strings_long = []
    for list_element in speaker_strings:
        if len(list_element) >= 100:
            speaker_strings_long.append(list_element)
    else:
        pass
    
    # Put it in a string for export
    speaker_strings_long = '_'.join(speaker_strings_long)

    # Create an output file and put "text" in there
    file = open(output_filename, "w")
    file.write(speaker_strings_long)
    fileroot, fileext = os.path.splitext(raw_text)
    output_filename = fileroot+'_clean.txt'
    file.close()   


KeyboardInterrupt: 

## Compile all .txt file data into a dataframe

In [114]:
cleantxt_list = glob.glob("Austin_Transcripts/*_clean.txt")
text_dict = {}
date_list, text_list = list(), list()
for filename in cleantxt_list:
    with open(filename, 'r') as f:
        data = f.read()
    date = filename.split("/")[1][:-10]
    for element in data.split('_'):
        text_list.append(element)
        date_list.append(date)
    #key = filename.split("/")[1]
    #date_list.append(key[:-10])
    #text_list.append(data)
    #text_dict[key[:-10]] = data


In [113]:
len(data.split('_'))

237

In [106]:
filename.split('/')[1][:-9]

'2019_08_08_'

In [115]:
text_dict = {'date': date_list, 'text': text_list}
df = pd.DataFrame(text_dict)

In [116]:
df.head()

date                                               text
0  2015_01_29   Okay and 14 14 is the relocation assistance a...
1  2015_01_29   17 Okay yes 17 appropriating 170000 grant fun...
2  2015_01_29   Thank you may you mayor pro tem and councilme...
3  2015_01_29   They very quickly will do this Theyll take th...
4  2015_01_29   Thank you members Matthew assistant attorney ...

In [117]:
df.shape

(13509, 2)

In [118]:
len(cleantxt_list)

57

In [96]:
list(text_dict.keys())[:2]

['2015_01_29', '2015_02_26']

In [99]:
type(text_dict['2015_01_29'])

str

In [91]:
pd.DataFrame(text_dict, index = range(len(cleantxt_list)))

2015_01_29  \
0    Okay and 14 14 is the relocation assistance a...   
1    Okay and 14 14 is the relocation assistance a...   
2    Okay and 14 14 is the relocation assistance a...   
3    Okay and 14 14 is the relocation assistance a...   
4    Okay and 14 14 is the relocation assistance a...   
5    Okay and 14 14 is the relocation assistance a...   
6    Okay and 14 14 is the relocation assistance a...   
7    Okay and 14 14 is the relocation assistance a...   
8    Okay and 14 14 is the relocation assistance a...   
9    Okay and 14 14 is the relocation assistance a...   
10   Okay and 14 14 is the relocation assistance a...   
11   Okay and 14 14 is the relocation assistance a...   
12   Okay and 14 14 is the relocation assistance a...   
13   Okay and 14 14 is the relocation assistance a...   
14   Okay and 14 14 is the relocation assistance a...   
15   Okay and 14 14 is the relocation assistance a...   
16   Okay and 14 14 is the relocation assistance a...   
17   Okay and 14 14 is the relocation assistance a...   
18   Okay and 14 14 is the relocation assistance a...   
19   Okay and 14 14 is the relocation assistance a...   
20   Okay and 14 14 is the relocation assistance a...   
21   Okay and 14 14 is the relocation assistance a...   
22   Okay and 14 14 is the relocation assistance a...   
23   Okay and 14 14 is the relocation assistance a...   
24   Okay and 14 14 is the relocation assistance a...   
25   Okay and 14 14 is the relocation assistance a...   
26   Okay and 14 14 is the relocation assistance a...   
27   Okay and 14 14 is the relocation assistance a...   
28   Okay and 14 14 is the relocation assistance a...   
29   Okay and 14 14 is the relocation assistance a...   
30   Okay and 14 14 is the relocation assistance a...   
31   Okay and 14 14 is the relocation assistance a...   
32   Okay and 14 14 is the relocation assistance a...   
33   Okay and 14 14 is the relocation assistance a...   
34   Okay and 14 14 is the relocation assistance a...   
35   Okay and 14 14 is the relocation assistance a...   
36   Okay and 14 14 is the relocation assistance a...   
37   Okay and 14 14 is the relocation assistance a...   
38   Okay and 14 14 is the relocation assistance a...   
39   Okay and 14 14 is the relocation assistance a...   
40   Okay and 14 14 is the relocation assistance a...   
41   Okay and 14 14 is the relocation assistance a...   
42   Okay and 14 14 is the relocation assistance a...   
43   Okay and 14 14 is the relocation assistance a...   
44   Okay and 14 14 is the relocation assistance a...   
45   Okay and 14 14 is the relocation assistance a...   
46   Okay and 14 14 is the relocation assistance a...   
47   Okay and 14 14 is the relocation assistance a...   
48   Okay and 14 14 is the relocation assistance a...   
49   Okay and 14 14 is the relocation assistance a...   
50   Okay and 14 14 is the relocation assistance a...   
51   Okay and 14 14 is the relocation assistance a...   
52   Okay and 14 14 is the relocation assistance a...   
53   Okay and 14 14 is the relocation assistance a...   
54   Okay and 14 14 is the relocation assistance a...   
55   Okay and 14 14 is the relocation assistance a...   
56   Okay and 14 14 is the relocation assistance a...   

                                           2015_02_26  \
0    Okay and 14 14 is the relocation assistance a...   
1    Okay and 14 14 is the relocation assistance a...   
2    Okay and 14 14 is the relocation assistance a...   
3    Okay and 14 14 is the relocation assistance a...   
4    Okay and 14 14 is the relocation assistance a...   
5    Okay and 14 14 is the relocation assistance a...   
6    Okay and 14 14 is the relocation assistance a...   
7    Okay and 14 14 is the relocation assistance a...   
8    Okay and 14 14 is the relocation assistance a...   
9    Okay and 14 14 is the relocation assistance a...   
10   Okay and 14 14 is the relocation assistance a...   
11   Okay and 14 14 is the relocation assistance a..

In [84]:
cleantxt_list = glob.glob("Austin_Transcripts/*_clean.txt")

li = []

for filename in cleantxt_list:
    df = pd.read_txt(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

AttributeError: module 'pandas' has no attribute 'read_txt'

In [83]:
frame.head()

Empty DataFrame
Columns: [ Okay and 14 14 is the relocation assistance and authorizing the payment of relocation funds located at 1127 at 52nd street I know we have an exce ption department I know theres been complaints I know well inspect the units for deficiencies and problematic issues I was wondering why this did not occur quicker I had a phone call last year about this property Unfortunately I was in the hospital again so I was not able to address it with you But in the future thats more pertinent and more efficient in listening to the complaints of the resident I understand that the owner has been cited or will be cited But thats not enough Number 15 is th e Austin community college increased funding for child care equality Any increase for child care equality is a no brainer Child  children  child needs health care and theyre not getting it A lot of people in east Austin A lot of children are not g etting top care at the childrens hospital Ive seen it They say call the mayor Thats your job You said number 16 Number 16 is just a increasing for the community network and Raul is the the executive director of the organization Called the communit y action network Thats money well spent for the community And again mayor Number 16 _ 17 Okay yes 17 appropriating 170000 grant funds for the Austin shelter Always supportive of women and especially children who ar e at risk These children are at risk mayor council They need this funding for actually appropriate needs immediate needs So this is an appropriate  appropriate funding for the Austin children for women and children Ill always support them You sa id number 18 right Number 18 has to do with increased mentoring is a no brainer Former teacher at ACC Austin independent school district And I taught also juniors in college But mentoring is very important to get the kids back on tra ck on education and the last item mayor And allow me to  Usa today good article about homeless veterans in Austin Texas Coming to Austin because he said he knows for a fact that there are more than 5000 homeless veterans in Austin Texas FYI I like to work with you and your office on that if it would be okay _ Thank you may you mayor pro tem and councilmembers Im David king Speaking on item 27 regarding the city manager search advisory task force I think its important task force Glad youre taking an initiative to make sure the community is involved i n the process I think youre suggesting that I believe this will happen The task force the membership reflecting the diversity of our community and I hope you will allow the task force to comply with the Texas open meetings laws and not allow lobbyists to be appointed to this important task force Thank you very much for listening to my comments _ They very quickly will do this Theyll take the consent agenda Well ask people if they did the full three minutes Andy young will get called when we have item 26 The items pulled now are 9 10 14 19 22 24 25 and 26 Is there a motion to approve the consent agenda Ms Houston makes that motion Is there a second Mr Casar any discussion _ Thank you members Matthew assistant attorney I wan t to approve a settlement amount of 250000 in the city of Austin lawsuit As discussed in executive session on April 4 this lawsuit is related to an automobile versus pedestrian incident that occurred on January 26 2016 As a result of the accident sh e suffered significant injuries In exchange for the settlement the city will obtain a full and final agreement that releases the city and city employees from claims that were or could have been asserted in the lawsuit The law department recommends that you approve this payment of the settlement amount pursuant to these terms Any questions _ Is there a motion to approve item 9 Councilmember Garza is our second in this Councilmember Renteria Any discussion Those in favor of the proposed settlement please raise your hand Those opposed Unanimous on the dias with Ms Poole off Thank you very much _

In [73]:
## testing gensim keywords
from gensim.summarization import keywords
text = '''17 Okay yes 17 appropriating 170000 grant funds for the Austin shelter Always supportive of women and especially children who ar e at risk These children are at risk mayor council They need this funding for actually appropriate needs immediate needs So this is an appropriate  appropriate funding for the Austin children for women and children Ill always support them You sa id number 18 right Number 18 has to do with increased mentoring is a no brainer Former teacher at ACC Austin independent school district And I taught also juniors in college But mentoring is very important to get the kids back on tra ck on education and the last item mayor And allow me to  Usa today good article about homeless veterans in Austin Texas Coming to Austin because he said he knows for a fact that there are more than 5000 homeless veterans in Austin Texas FYI I like to work with you and your office on that if it would be okay'''
keywords(text).split('\n')

['appropriating',
 'appropriate',
 'austin',
 'mayor',
 'texas',
 'children',
 'today']

## Basic keyword maker

In [18]:
#print(keywords)